In [4]:
from __future__ import annotations

import gzip
import json
import subprocess
from pathlib import Path

import numpy as np
import pandas as pd
import yaml
from rich.prompt import Prompt
from yaml.loader import SafeLoader

In [5]:
events_asc_file = "/Users/julia/Desktop/NOWA.nosync/eyelink/2eyes/sub-99_task-FreeView_run-01_eyeData_events.asc"

In [6]:
def _load_asc_file(events_asc_file: str | Path) -> list[str]:
    with open(events_asc_file) as f:
        return f.readlines()


def _load_asc_file_as_df(events_asc_file: str | Path) -> pd.DataFrame:
    # dataframe for events, all
    events = _load_asc_file(events_asc_file)
    return pd.DataFrame([ms.split() for ms in events if ms.startswith("MSG")])


def _load_asc_file_as_reduced_df(events_asc_file: str | Path) -> pd.DataFrame:
    # reduced dataframe without MSG and sample columns
    df_ms = _load_asc_file_as_df(events_asc_file)
    return pd.DataFrame(df_ms.iloc[0:, 2:])

In [7]:
events = _load_asc_file(events_asc_file)
df_ms = _load_asc_file_as_df(events_asc_file)
df_ms_reduced = _load_asc_file_as_reduced_df(events_asc_file)

In [8]:
def _calibrations(df: pd.DataFrame) -> pd.DataFrame:
    return df[df[3] == "CALIBRATION"]

In [9]:
def _extract_CalibrationType(df: pd.DataFrame) -> list[int]:
    return _calibrations(df).iloc[0:1, 2:3].to_string(header=False, index=False)

In [10]:
def _extract_CalibrationCount(df: pd.DataFrame) -> int:
    if _extract_RecordedEye(df) == "Both":
        return len(_calibrations(df)) // 2
    return len(_calibrations(df))

In [11]:
def _get_calibration_positions(df: pd.DataFrame) -> list[int]:
    if _extract_RecordedEye(df) == "Both":
        return (
            np.array(df[df[2] == "VALIDATE"][8].str.split(",", expand=True))
            .astype(int)
            .tolist()
        )[::2]
    return (
        np.array(df[df[2] == "VALIDATE"][8].str.split(",", expand=True))
        .astype(int)
        .tolist()
    )

In [12]:
def _extract_CalibrationPosition(df: pd.DataFrame) -> list[list[int]]:
    cal_pos = _get_calibration_positions(df)
    cal_num = len(cal_pos) // _extract_CalibrationCount(df)

    CalibrationPosition: list[list[int]] = []

    if len(cal_pos) == 0:
        return CalibrationPosition

    CalibrationPosition.extend(
        cal_pos[i : i + cal_num] for i in range(0, len(cal_pos), cal_num)
    )
    return CalibrationPosition

In [13]:
def _extract_CalibrationUnit(df: pd.DataFrame) -> str:
    if len(_get_calibration_positions(df)) == 0:
        return ""

    cal_unit = (
        (df[df[2] == "VALIDATE"][[13]])
        .iloc[0:1, 0:1]
        .to_string(header=False, index=False)
    )
    if cal_unit == "pix.":
        return "pixel"
    elif cal_unit in ["cm", "mm"]:
        return cal_unit
    return ""

In [14]:
def _extract_EyeTrackingMethod(events: list[str]) -> str:
    return (
        pd.DataFrame(
            " ".join([tm for tm in events if tm.startswith(">>>>>>>")])
            .replace(")", ",")
            .split(",")
        )
        .iloc[1:2]
        .to_string(header=False, index=False)
    )

In [15]:
def _extract_DeviceSerialNumber(events: list[str]) -> str:
    return (
        " ".join([sl for sl in events if sl.startswith("** SERIAL NUMBER:")])
        .replace("** SERIAL NUMBER: ", "")
        .replace("\n", "")
    )

In [16]:
def _extract_PupilFitMethod(df: pd.DataFrame) -> str:
    return (df[df[2] == "ELCL_PROC"]).iloc[0:1, 1:2].to_string(header=False, index=False)

In [17]:
def _extract_SamplingFrequency(df: pd.DataFrame) -> int:
    return int(df[df[2] == "RECCFG"].iloc[0:1, 2:3].to_string(header=False, index=False))

In [18]:
def _validations(df: pd.DataFrame) -> pd.DataFrame:
    return df[df[3] == "VALIDATION"]

In [19]:
def _has_validation(df: pd.DataFrame) -> bool:
    return not _validations(df).empty

In [20]:
def _extract_ManufacturersModelName(events: list[str]) -> str:
    return (
        " ".join([ml for ml in events if ml.startswith("** EYELINK")])
        .replace("** ", "")
        .replace("\n", "")
    )

In [21]:
def _extract_ScreenResolution(df: pd.DataFrame) -> list[int]:
    list_res = (
        (df[df[2] == "GAZE_COORDS"])
        .iloc[0:1, 3:5]
        .to_string(header=False, index=False)
        .replace(".00", "")
        .split(" ")
    )
    return [eval(i) for i in list_res]

In [22]:
def _extract_TaskName(events: list[str]) -> str:
    return (
        " ".join([ts for ts in events if ts.startswith("** RECORDED BY")])
        .replace("** RECORDED BY ", "")
        .replace("\n", "")
    )

In [23]:
def _extract_StartTime(events: list[str]) -> int:
    StartTime = (
        np.array(pd.DataFrame([st.split() for st in events if st.startswith("START")])[1])
        .astype(int)
        .tolist()
    )
    if len(StartTime) > 1:
        e2b_log.info(
            """Your input file contains multiple start times.\n
             As this is not seen as good practice in eyetracking experiments, \n
             only the first start time will be kept for the metadata file. \n
             Please consider changing your code accordingly
             for future eyetracking experiments.\n"""
        )
        return StartTime[0]
    return StartTime

In [24]:
def _extract_StopTime(events: list[str]) -> int:
    StopTime = (
        np.array(pd.DataFrame([so.split() for so in events if so.startswith("END")])[1])
        .astype(int)
        .tolist()
    )
    if len(StopTime) > 1:
        e2b_log.info(
            """Your input file contains multiple stop times.\n
             As this is not seen as good practice in eyetracking experiments, \n
             only the last stop time will be kept for the metadata file. \n
             Please consider changing your code accordingly
             for future eyetracking experiments.\n"""
        )
        return StopTime[-1]
    return StopTime

In [25]:

def _extract_RecordedEye(df: pd.DataFrame) -> str:
    eye = df[df[2] == "RECCFG"].iloc[0:1, 5:6].to_string(header=False, index=False)
    if eye == "L":
        return "Left"
    elif eye == "R":
        return "Right"
    elif eye == "LR":
        return ["Left", "Right"]
    return ""

In [26]:
DeviceSerialNumber = _extract_DeviceSerialNumber(events)
EyeTrackingMethod = _extract_EyeTrackingMethod(events)
ManufacturersModelName = _extract_ManufacturersModelName(events)
CalibrationUnit = _extract_CalibrationUnit(df_ms_reduced)
CalibrationType = _extract_CalibrationType(df_ms_reduced)
PupilFitMethod = _extract_PupilFitMethod(df_ms_reduced)
SamplingFrequency = _extract_SamplingFrequency(df_ms_reduced)
StartTime = _extract_StartTime(events)
StopTime = _extract_StopTime(events)

print(DeviceSerialNumber, EyeTrackingMethod, ManufacturersModelName, CalibrationUnit, CalibrationType, PupilFitMethod, SamplingFrequency, StartTime, StopTime)

CLG-BAF38 P-CR EYELINK II CL v5.12 May 12 2017 pixel HV13 CENTROID 1000 [767979] [819652]


In [65]:
def _2eyesmode(df: pd.DataFrame) -> bool:
    eye = df[df[2] == "RECCFG"].iloc[0:1, 5:6].to_string(header=False, index=False)
    if eye == "LR":
        two_eyes = True
    return two_eyes

In [66]:
_2eyesmode(df_ms_reduced)


True

In [74]:
def _extract_RecordedEye(df: pd.DataFrame) -> str:
    eye = df[df[2] == "RECCFG"].iloc[0:1, 5:6].to_string(header=False, index=False)
    if eye == "L":
        return eye1
    elif eye == "R":
        return "Right"
    elif eye == "LR":
        return ["Left", "Right"]
    return "" 

In [94]:
def _extract_AverageCalibrationError(df: pd.DataFrame) -> list[float]:
    if not _has_validation(df):
        return []
    if _extract_CalibrationCount(df) > 1:
        return np.array(_validations(df)[[9]]).astype(float).tolist()
    return np.array(_validations(df)[[9]]).astype(float).tolist()

In [92]:
def _extract_MaximalCalibrationError(df: pd.DataFrame) -> list[float]:
    if not _has_validation(df):
        return []
    return np.array(_validations(df)[[11]]).astype(float).tolist()

In [101]:
if _2eyesmode(df_ms_reduced) == True:
        json_eye1 = {
                "RecordedEye": (_extract_RecordedEye(df_ms_reduced)[0]),
                "AverageCalibrationError": (_extract_AverageCalibrationError(df_ms)[0::2]),
                "MaximalCalibrationError": (_extract_MaximalCalibrationError(df_ms)[0::2])
        }
        json_eye2 = {
                "RecordedEye": (_extract_RecordedEye(df_ms_reduced)[1]),
                "AverageCalibrationError": (_extract_AverageCalibrationError(df_ms)[1::2]),
                "MaximalCalibrationError": (_extract_MaximalCalibrationError(df_ms)[1::2])
        }
else: json_eye1 = {
                "RecordedEye": _extract_RecordedEye(df_ms_reduced),
                "AverageCalibrationError": _extract_AverageCalibrationError(df_ms),
                "MaximalCalibrationError": _extract_MaximalCalibrationError(df_ms)
        }


In [98]:
json_eye1

{'RecordedEye': 'Left',
 'AverageCalibrationError': [[0.29]],
 'MaximalCalibrationError': [[0.62]]}

In [99]:
json_eye2

{'RecordedEye': 'Right',
 'AverageCalibrationError': [[0.35]],
 'MaximalCalibrationError': [[1.21]]}

In [103]:
base_json = {
            "Manufacturer": "SR-Research",
            "DeviceSerialNumber": _extract_DeviceSerialNumber(events),
            "EyeTrackingMethod": _extract_EyeTrackingMethod(events),
            "ManufacturersModelName": _extract_ManufacturersModelName(events),
            "CalibrationUnit": _extract_CalibrationUnit(df_ms_reduced),
            "CalibrationType": _extract_CalibrationType(df_ms_reduced),
            "PupilFitMethod": _extract_PupilFitMethod(df_ms_reduced),
            "SamplingFrequency": _extract_SamplingFrequency(df_ms_reduced),
            "StartTime": _extract_StartTime(events),
            "StopTime": _extract_StopTime(events),
        }

In [106]:
if _2eyesmode(df_ms_reduced) == True:
        metadata_eye1 = {
                "RecordedEye": (_extract_RecordedEye(df_ms_reduced)[0]),
                "AverageCalibrationError": (_extract_AverageCalibrationError(df_ms)[0::2]),
                "MaximalCalibrationError": (_extract_MaximalCalibrationError(df_ms)[0::2])
        }
        metadata_eye2 = {
                "RecordedEye": (_extract_RecordedEye(df_ms_reduced)[1]),
                "AverageCalibrationError": (_extract_AverageCalibrationError(df_ms)[1::2]),
                "MaximalCalibrationError": (_extract_MaximalCalibrationError(df_ms)[1::2])
        }
else: 
    metadata_eye1 = {
            "RecordedEye": _extract_RecordedEye(df_ms_reduced),
            "AverageCalibrationError": _extract_AverageCalibrationError(df_ms),
            "MaximalCalibrationError": _extract_MaximalCalibrationError(df_ms)
    }

In [108]:
json_eye1 = base_json | metadata_eye1
json_eye1

{'Manufacturer': 'SR-Research',
 'DeviceSerialNumber': 'CLG-BAF38',
 'EyeTrackingMethod': 'P-CR',
 'ManufacturersModelName': 'EYELINK II CL v5.12 May 12 2017',
 'CalibrationUnit': 'pixel',
 'CalibrationType': 'HV13',
 'PupilFitMethod': 'CENTROID',
 'SamplingFrequency': 1000,
 'StartTime': [767979],
 'StopTime': [819652],
 'RecordedEye': 'Left',
 'AverageCalibrationError': [[0.29]],
 'MaximalCalibrationError': [[0.62]]}

In [109]:
def generate_output_filename(
    output_dir: Path, input_file: Path, suffix: str, extension: str
) -> Path:
    """Generate output filename."""
    filename = Path(input_file).stem
    if filename.endswith(suffix):
        suffix = ""
    return output_dir / f"{filename}{suffix}.{extension}"

In [113]:
if _2eyesmode == True:
    output_filename_eye1 = generate_output_filename(
            output_dir=output_dir, input_file=input_file, suffix="recording-eye1_physio", extension="json"
        )
    with open(output_filename_eye1, "w") as outfile:
        json.dump(json_eye1, outfile, indent=4)

    output_filename_eye2 = generate_output_filename(
            output_dir=output_dir, input_file=input_file, suffix="recording-eye2_physio", extension="json"
        )
    with open(output_filename_eye2, "w") as outfile:
        json.dump(json_eye2, outfile, indent=4)
    
    #e2b_log.info(f"files generated: {output_filename_eye1} and {output_filename_eye2}")

else:
    output_filename_eye1 = generate_output_filename(
            output_dir=output_dir, input_file=input_file, suffix="recording-eye1_physio", extension="json"
        )
    with open(output_filename_eye1, "w") as outfile:
        json.dump(json_eye1, outfile, indent=4)
    
    #e2b_log.info(f"file generated: {output_filename_eye1}")


